# Analyzing Short-Term Hospital Readmissions Using the MIMIC-III Dataset

## Data Preprocessing 

The purpose of this notebook is to create a unified dataframe from disparate .csv files in the MIMIC-III dataset for the purposes of feature extraction and subsequent analysis.

In [24]:
import pandas as pd
import numpy as np

In [25]:
data_path = '../data/mimic-iii/'
output = '../data/pre-processed/'

In [26]:
# Read in relevant data for our project

admissions = pd.read_csv(data_path + 'ADMISSIONS.csv/ADMISSIONS.csv')
patients = pd.read_csv(data_path + 'PATIENTS.csv/PATIENTS.csv')
diagnoses = pd.read_csv(data_path + 'DIAGNOSES_ICD.csv/DIAGNOSES_ICD.csv')
diagnosis_dict = pd.read_csv(data_path + 'D_ICD_DIAGNOSES.csv/D_ICD_DIAGNOSES.csv')

In [27]:
# Convert datetime field types
admissions['ADMITTIME'] = pd.to_datetime(admissions['ADMITTIME'])
admissions['DISCHTIME'] = pd.to_datetime(admissions['DISCHTIME'])
admissions['DEATHTIME'] = pd.to_datetime(admissions['DEATHTIME'])
patients['DOB'] = pd.to_datetime(patients['DOB'])

In [28]:
# Join the patients and admissions table on SUBJECT_ID (patient unique ID)
patient_admissions = pd.merge(
    admissions, 
    patients,
    on='SUBJECT_ID',
    how='left'
)

### Excluding Patients by Age

Because children, newborns and the very elderly exhibit some different challenges for medical professionals, we'll seek to exclude those patients for the purposes of this analysis. 

We know that patients that were 89 and over were handled with a special de-identification process to further obscure their age and protect their identity. These patients were made to look 300 years old on the date of their first admission within the dataset. We plan to exclude them, but direct calculations of their age will give us some trouble, so we will first address those patients whose DOB has been age corrected, then calculate the age for the rest of the patient population, and finally filter our results for a minimum age of 18, and a maximum age of 88. 

In [29]:
# Set upper and lower bounds for age
MIN_AGE = 18
MAX_AGE = 89

# Initialize column
patient_admissions['AGE_AT_ADMISSION'] = np.nan

elderly_mask = patient_admissions['DOB'].dt.year <= 1923

patient_admissions.loc[elderly_mask, 'AGE_AT_ADMISSION'] = MAX_AGE

remaining_population = patient_admissions['AGE_AT_ADMISSION'].isna()

patient_admissions.loc[remaining_population, 'AGE_AT_ADMISSION'] = \
(patient_admissions.loc[remaining_population, 'ADMITTIME'] - patient_admissions.loc[remaining_population, 'DOB']).dt.days / 365.25


patient_admissions_filtered = patient_admissions[
    (patient_admissions['AGE_AT_ADMISSION'] >= MIN_AGE) &
    (patient_admissions['AGE_AT_ADMISSION'] < MAX_AGE)
].copy()

print(f"Unfiltered: {len(patient_admissions)}")
print(f"Filtered for age {MIN_AGE} - {MAX_AGE}: {len(patient_admissions_filtered)}")

Unfiltered: 58976
Filtered for age 18 - 89: 48149


### Excluding patients who died on their first visit

We seek to examine factors leading to re-admission. If a patient dies on their first visit, they can not be readmitted to the hospital later, so we'll exclude them from our analysis.


In [30]:
# Mask to identify death on first admission
death_during_first_stay = (
    patient_admissions_filtered['DEATHTIME'].notna() &
    (patient_admissions_filtered['DEATHTIME'] >= patient_admissions_filtered['ADMITTIME']) &
    (patient_admissions_filtered['DEATHTIME'] <= patient_admissions_filtered['DISCHTIME'])
)

# Use the mask to filter out patients who died on first admission
patient_admissions_filtered = patient_admissions_filtered[~death_during_first_stay]

print(f"Filtered for death on first stay: {len(patient_admissions_filtered)}")

Filtered for death on first stay: 43032


### Bringing in Diagnoses

To get the Diagnosis for each patient in plain English, we will need to make use of the ICD-9 dictionary which defines codes for conditions

In [31]:
diagnoses_full = pd.merge(
    diagnoses,
    diagnosis_dict,
    on='ICD9_CODE',
    how='left'
)

We don't need to retain diagnoses for patients we have already excluded, so we can filter those out before doing any more intensive operations.

In [32]:
selected_ids = patient_admissions_filtered['HADM_ID'].unique()
relevant_diagnoses = diagnoses_full[diagnoses_full['HADM_ID'].isin(selected_ids)]

### Comorbidities 

Comorbidities are the simultaneous presence of two or more diseases or medical conditions. They are important to measure, as they represent confounding factors and a level of difficulty for doctors to assess. We will use the `comorbidipy` package to obtain a Charlson score for comorbidities.

In [33]:
from comorbidipy import comorbidity

charlson = comorbidity(
    df=relevant_diagnoses,
    id ='HADM_ID',
    code='ICD9_CODE',
    age=None,
    score='charlson',
    icd='icd9'    
)

We now have the comorbidity scores for each patient, with each comorbidity being given a binary score of 0 or 1, and the total score being the sum of all comorbidities the patient has. We only care about the total score, so we will keep that and merge it into the `patient_admissions_filtered` dataframe

In [34]:
charlson = charlson[['HADM_ID', 'comorbidity_score']]

In [35]:
patient_admissions_filtered = pd.merge(
    patient_admissions_filtered, 
    charlson,
    on='HADM_ID',
    how='left'
)

### Adding in Length of stay

Length of stay is a simple calculation of discharge time minus admission time (`ADMITTIME` - `DISCHTIME`)

In [36]:
patient_admissions_filtered['LENGTH_OF_STAY'] = (patient_admissions_filtered['DISCHTIME'] - patient_admissions_filtered['ADMITTIME']).dt.days

### Determining Re-admissions

A standard timeframe for monitoring readmissions is 30 days. To figure out if a patient has been readmitted during this timeframe, first we want to see if each particular patient has been readmitted at all. If so, we can determine when they were discharged and store that in a column for previous discharge time `PREV_DISCHTIME`. Then we can calculate the difference between the next admission and the previous discharge. We convert that timedelta to days and store it a new column for days to readmission `DAYS_TO_READMIT`. Finally, we flag any values that are within our 30 day window and store that in a binary column `READMISSION_FLAG`.

In [37]:
patient_admissions_filtered = patient_admissions_filtered.sort_values(by=['SUBJECT_ID','ADMITTIME'])
patient_admissions_filtered['PREV_DISCHTIME'] = patient_admissions_filtered.groupby('SUBJECT_ID')['DISCHTIME'].shift(1)
patient_admissions_filtered['TIME_TO_READMIT'] = (patient_admissions_filtered['ADMITTIME'] - patient_admissions_filtered['PREV_DISCHTIME'])
patient_admissions_filtered['DAYS_TO_READMIT'] = patient_admissions_filtered['TIME_TO_READMIT'].dt.days

In [38]:
READMISSION_WINDOW = 30

patient_admissions_filtered['READMISSION_FLAG'] = (
    (patient_admissions_filtered['DAYS_TO_READMIT'].notna()) &
    (patient_admissions_filtered['DAYS_TO_READMIT'] <= READMISSION_WINDOW)
).astype(int)

### Accounting for Planned admissions

In some cases, a patient may return to the hospital voluntarily. Perhaps they had a visit just before an elective surgery. In order to not overestimate our readmission numbers, we can account for this using the `ADMISSION_TYPE` column. 

In [39]:
patient_admissions_filtered.loc[patient_admissions_filtered['ADMISSION_TYPE'] == 'ELECTIVE', 'READMISSION_FLAG'] = 0

In [40]:
patient_admissions_filtered.columns

Index(['ROW_ID_x', 'SUBJECT_ID', 'HADM_ID', 'ADMITTIME', 'DISCHTIME',
       'DEATHTIME', 'ADMISSION_TYPE', 'ADMISSION_LOCATION',
       'DISCHARGE_LOCATION', 'INSURANCE', 'LANGUAGE', 'RELIGION',
       'MARITAL_STATUS', 'ETHNICITY', 'EDREGTIME', 'EDOUTTIME', 'DIAGNOSIS',
       'HOSPITAL_EXPIRE_FLAG', 'HAS_CHARTEVENTS_DATA', 'ROW_ID_y', 'GENDER',
       'DOB', 'DOD', 'DOD_HOSP', 'DOD_SSN', 'EXPIRE_FLAG', 'AGE_AT_ADMISSION',
       'comorbidity_score', 'LENGTH_OF_STAY', 'PREV_DISCHTIME',
       'TIME_TO_READMIT', 'DAYS_TO_READMIT', 'READMISSION_FLAG'],
      dtype='object')

In [41]:
patient_admissions_filtered = patient_admissions_filtered.drop(['ROW_ID_x',
                                                                'ADMISSION_LOCATION', 
                                                                'DISCHARGE_LOCATION', 
                                                                'EDREGTIME',
                                                                'EDOUTTIME',
                                                                'DIAGNOSIS',
                                                                'HOSPITAL_EXPIRE_FLAG',
                                                                'HAS_CHARTEVENTS_DATA',
                                                                'ROW_ID_y',
                                                                'DOD',
                                                                'DOD_HOSP',
                                                                'DOD_SSN',
                                                                'EXPIRE_FLAG',
                                                                'TIME_TO_READMIT'
                                                                ],
                                                               axis=1
                                                            )

In [42]:
patient_admissions_filtered['ETHNICITY'] = patient_admissions_filtered['ETHNICITY'].replace({
    'WHITE - RUSSIAN': 'WHITE',
    'WHITE - OTHER EUROPEAN': 'WHITE',
    'WHITE - BRAZILIAN': 'WHITE',
    'WHITE - EASTERN EUROPEAN': 'WHITE',
    'BLACK/AFRICAN AMERICAN': 'BLACK',
    'BLACK/AFRICAN': 'BLACK',
    'BLACK/CAPE VERDEAN': 'BLACK',
    'HISPANIC/LATINO - PUERTO RICAN': 'HISPANIC/LATINO',
    'HISPANIC/LATINO - DOMINICAN REPUBLIC': 'HISPANIC/LATINO',
    'HISPANIC/LATINO - SALVADORAN': 'HISPANIC/LATINO',
    'HISPANIC/LATINO - GUATEMALAN': 'HISPANIC/LATINO',
    'HISPANIC/LATINO - COLUMBIAN': 'HISPANIC/LATINO',
    'ASIAN - CHINESE': 'ASIAN',
    'ASIAN - ASIAN INDIAN': 'ASIAN',
    'ASIAN - VIETNAMESE': 'ASIAN',
    'ASIAN - CAMBODIAN': 'ASIAN',
    'UNKNOWN/NOT SPECIFIED': 'OTHER/UNKNOWN',
    'OTHER': 'OTHER/UNKNOWN',
    'MULTI RACE ETHNICITY': 'OTHER/UNKNOWN',
    'PORTUGUESE': 'HISPANIC/LATINO', 
    'AMERICAN INDIAN/ALASKA NATIVE': 'INDIGENOUS',
    'NATIVE HAWAIIAN OR OTHER PACIFIC ISLANDER': 'INDIGENOUS',
    'MIDDLE EASTERN': 'MIDDLE EASTERN',
    'SOUTH AMERICAN': 'HISPANIC/LATINO'
}
)

In [43]:
pd.to_pickle(patient_admissions_filtered, output + 'patient_admissions')